# Natural Language Processing with Deep Learning

* what is NLP - understand human language or generate text or sound which humans can understand
* why it's difficult - Sparsity of categoricals
* word embedding - [skipgram & cbow](https://www.tensorflow.org/tutorials/representation/word2vec)
* RNN - $ h_t = \phi(Ux_t + Wh_{t-1}) $

### Libraries
* [NLTK](https://pythonspot.com/category/nltk/)
* [SpaCy](https://spacy.io/usage/)
* tensorflow and keras

### Lecture
[stanford NLP yt](https://www.youtube.com/watch?v=OQQ-W_63UgQ&list=PL3FW7Lu3i5Jsnh1rnUwq_TcylNr7EkRe6)

### Applications
* Sentiment analysis
* Translation
* Text summarization
* Coreference resolution
* and more

### Sentiment analysis on IMDB dataset

In [7]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import *
from keras.layers import LSTM
from keras.datasets import imdb

# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)


#### change the model, experiment

In [8]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

Build model...


In [9]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 73s 3ms/step - loss: 0.4041 - acc: 0.8006 - val_loss: 0.3172 - val_acc: 0.8627
Epoch 2/2
25000/25000 [==============================] - 58s 2ms/step - loss: 0.2293 - acc: 0.9095 - val_loss: 0.3222 - val_acc: 0.8648


In [10]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 11s 431us/step
Test score: 0.3221742620754242
Test accuracy: 0.86484


### Text classification in 20 newsgroup dataset
[example](https://github.com/keras-team/keras/blob/master/examples/pretrained_word_embeddings.py)

### Text generation from Nietzsche's writings
[example](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py)

In [11]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=40,
callbacks=[print_callback])

606208/600901 [==============================] - 15s 25us/step
corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...
Epoch 1/40
200285/200285 [==============================] - 276s 1ms/step - loss: 1.9845

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "beneficial and grateful as those delusio"
beneficial and grateful as those delusion, and and the sense and and and in the proseration, and and any and the moral and any the stance of the probaring and and and any the restradition, and a the stands and the moral and and any and the stands of the strong and any a consitions, and a probaring that is moral and any action of the love and a moral and and and any his moral inderitate and any a perhaps and any and a probarinateral and 
----- diversity: 0.5
----- Generating with seed: "beneficial and grateful as those delusio"
beneficial and grateful as those delusions of his wortion
that who voly of moral the power, an

the and the sense, the his womps the propourable laws to the
----- diversity: 1.0
----- Generating with seed: "an wishes to be independent, and therefo"
an wishes to be independent, and therefore basking to communist what you hards to me, rilase of the flatther of these torarial and part to the head thought-egrause; and suthance of extesiably as i a himusther natural the ploarents an his
a comparem, an superto sufferent, the farths some thme hearts the aposic may
justice, and was that
a new seunalivate, any art excher in order to
the gradiations and
with way has wele to nothing and
darr
----- diversity: 1.2
----- Generating with seed: "an wishes to be independent, and therefo"
an wishes to be independent, and therefore, naturl casition topes imadly comman cyrter has alpost rials self
have not only that our proxt wakegleryly not religious be
!

da"gmost herder, and
th these as a regard in herewthen, untor believis,
morbhadesly on dispreasev opicioul a sincemst is place?

"though : suall

ases of the martyrdom of knowledge about the stands and any any a strength the constranted to the stands of the stands and the stands and only the proced to the stands and a stands the stands and any as the stands and the stands and any any intercaled to the stands and a stands and an actions of the strength of the stands and intercaled to the stands of the spiritual the morality of the stands and a strength of the stands and stands of 
----- diversity: 0.5
----- Generating with seed: "ases of the martyrdom of knowledge about"
ases of the martyrdom of knowledge about understood stands the approxionation to stands and words and is the
the about its all the himself as the "been in himself in in the stands of desires of the morality, with women in the religious to be the the stapty and of the chard regard of man consideration to a power at the delicate still with the superior to the appeared to false the prain strange the contradically and who was to present ant
----- diversity: 1.0
-----

do un afute of man like themselves trans, manty was
same , he they not mons organs, and instance, that no o
----- diversity: 1.2
----- Generating with seed: " in every
scheme of ascetic ethics, man "
 in every
scheme of ascetic ethics, man are beate
fire that to iting alune)
upon guilt as it
oriean po hight
just, emplance
grevation well aortule it expsnwitned. al-filted as wealle.

a .tan
that hew
, should values or
-"think, fancce importy
apwor insiders like free-ocrevoming,
oibobs--whooligh, it has so horshed prepors amganire findder upon grances, scharmchar, for reader world which mean
for will to himsely certainly be acmanss, as
Epoch 12/40
200285/200285 [==============================] - 279s 1ms/step - loss: 1.3836

----- Generating text after Epoch: 11
----- diversity: 0.2
----- Generating with seed: "eaks of "the
whole true gospel in the lo"
eaks of "the
whole true gospel in the long of the sense of the sense of the consequences of the present of the superstition of the senses 

ensions or excessive expedients and lateration of the soul and and recoming the
action in the serious the portions of the self-and to the more complete interest, the distrust. the onleranding such and always conduct by the master of the own of the sentiment which were all the presented the end, that seems to which in the science that they have no higher a worded the intercting and in the end forth, of the principal more delight of the m
----- diversity: 1.0
----- Generating with seed: "ensions or excessive expedients and late"
ensions or excessive expedients and later of my
sensual of canscioulyer
of artness"! whises malisn..

1ires in its alvariny is their the whole happiness in it. the avecouration so somethings of hinger given
with warn he changes.--rearous man who malie ; all the self-every bediaid, the specled a possess--palomay a cosid out. commoter therein
so long of the germans of the thing conceptionity and most  very long poil-put
seriouss of its
co
----- diversity: 1.2
-----

...

a follor.sjugnium, for cause prokentarchimy who crease folers an europerable to woon.=may here that us derovest inmoralmen
hence good! the picit is -impappopttion of asfutness, they flow mor
Epoch 26/40
200285/200285 [==============================] - 276s 1ms/step - loss: 1.3259

----- Generating text after Epoch: 25
----- diversity: 0.2
----- Generating with seed: " to continue such
investigations? mankin"
 to continue such
investigations? mankind, and the sensible in the most strange to the sensible in the conception of the sensible in the truth of the spirit in the same the sensible in the sensible in the subject of the strange to the soul, and the subject of being the sublime to the contemplate the spirit, the sensible in the world in the sensible in the sensation and and more the soul, the sensible in the senses of the considered the 
----- diversity: 0.5
----- Generating with seed: " to continue such
investigations? mankin"
 to continue such
investigations? mankind, the conforma

a metaphysical philosopher still estame. the guilt! perhapso we have neception. the little toneral
like trour entains his saint they obsortion for things, preciselps, a new
inner mode of the
origin as it is an anviently take
to which we aspica" with european to falowornmely should not neparable of
by philosopher "when be constant beianryt hence itself scide will be is anressism is regular shows in cormico?
eyonumed an
----- diversity: 1.2
----- Generating with seed: "from art than from
a metaphysical philos"
from art than from
a metaphysical philosopher, who
had comingor among
a tibuleem hence then? even wines that he senses ordeally,
when he whys..

. 

25 
=mereoven, tershes calculain them.

38. there is an forge deulcred by this indiciately the virtuesness of noble non-lumatine--that luthica
whe "nemgotedednces: in order adontsoun proaws betwelly fre's "is
inkinds
had germans; as undery
agains
wh "delicately standing and
to "too meletyfu
Epoch 30/40
200285/200285 [==================

accordance with a systempial that the most such a profoundly and the same case of the morality in the senses of the process and the fact that he has and the subject of the street the philosopher of the strange to the strong and here, which is not be differenting that is the strong to the self-freed and the process is a probably be the profoundly and the process and finally and and the process to the same conscience of t
----- diversity: 0.5
----- Generating with seed: "s and actions in
accordance with a syste"
s and actions in
accordance with a system is a philosophers of the sight--is and the way the two has not and there is the best to the love that the man in the conditioned in the best as well to the subject and emperition of means of the modern to the world, the conscience of the sentiment of the fundamental standard and finesting connectes in the most instinct to make the spirit as the same philosophy, where the most interpretes of the 
----- diversity: 1.0
----- Generating with 

 q(lupcely, they so ever if a theocogonar.=--to them. oneker away when he, as eraocally essentially, ye somelimen or this relability of than individual finital victions"s when most develop as "sc
----- diversity: 1.2
----- Generating with seed: " been unskilled and unseemly methods for"
 been unskilled and unseemly methods for the
middled and
literally occasionsfulles. in theus and heart, back and , i hopianing to voice of aimatid-forced, place
year only
sprain passion of an imposession poncefting, repolsed in itself, this gensh peinsity; yet hers an exelnated norself mystentic bit "is enough
and
use of devedon. the scientifourability one an antimis--actedge imloud asree. unworl
asmaming for the met'e different emotion
Epoch 37/40
200285/200285 [==============================] - 224s 1ms/step - loss: 1.3066

----- Generating text after Epoch: 36
----- diversity: 0.2
----- Generating with seed: " have." so
reasons the mind of the sleep"
 have." so
reasons the mind of the sleep is a man 

foundation upon which the higher rational the personality, there is the such a different intellectual things the serious hendence of the notic and interesting the same with the problem the interprotinted and problem of god and conditions of a sort of the sort of the
former and a philosopher and as importance of an asserselfed and instances that the deferable to the head heart that it
was a so be to be the world for those in which the so
----- diversity: 1.0
----- Generating with seed: "foundation upon which the higher rationa"
foundation upon which the higher rational
morals. this, they will would not not virtuor to vivid, and worthly who can be regarded, of his self-mending
delicately happinesss of "necessary sen--it is looking of cruelty man, a complete lifaile)ful incisible
is discrace is the result than by this first accompanily
referenced;
overootly very garial, and was a cewarre.m distrand, so belan thic of sevetorian, happiness, with its fines mirorial
----- diversity: 1.2
-----

Epoch 40/40
200285/200285 [==============================] - 224s 1ms/step - loss: 1.3076

----- Generating text after Epoch: 39

----- diversity: 0.2

----- Generating with seed: "foundation upon which the higher rationa"

foundation upon which the higher rational and the problem of the sense of the art of the self-decreed and the sense of the deperded to the world of the more problem is a sort of the problem of the self-despension of the more conception of the former and the deperd to the most such a strong soul of the sensible to the depers and and a philosophers and problem of the sense of the personal something and something the soul, and only a probl

----- diversity: 0.5

----- Generating with seed: "foundation upon which the higher rationa"

foundation upon which the higher rational the personality, there is the such a different intellectual things the serious hendence of the notic and interesting the same with the problem the interprotinted and problem of god and conditions of a sort of the sort of the
former and a philosopher and as importance of an asserselfed and instances that the deferable to the head heart that it
was a so be to be the world for those in which the so

----- diversity: 1.0

----- Generating with seed: "foundation upon which the higher rationa"

foundation upon which the higher rational
morals. this, they will would not not virtuor to vivid, and worthly who can be regarded, of his self-mending
delicately happinesss of "necessary sen--it is looking of cruelty man, a complete lifaile)ful incisible
is discrace is the result than by this first accompanily
referenced;
overootly very garial, and was a cewarre.m distrand, so belan thic of sevetorian, happiness, with its fines mirorial

----- diversity: 1.2

----- Generating with seed: "foundation upon which the higher rationa"

foundation upon which the higher rationated man accidely (neception addiols is inritrate), strend burtupments. emptatheed of
standard day for advancese, and oals
of the specialized, and its corpining or to woully afpured them--it were smallh, it is domination--succesial capais for men wiselfl i sentences, welest vanity and idea, on the
natance of aspect oo to be for pechates; nothing which, as the might it independerly hoped by one, hon


### Visual Question Anwering - NLP and CV
[example](https://github.com/anantzoid/VQA-Keras-Visual-Question-Answering)